# English - French Translation


## Data Extracation
In this notebook I will focus on the the extraction of of the data. This data is text data comprised of multiple documents files in english as well the same documents in french. These documents are each in a compressed .gz format. We will be unwrapping the data and combining all our data.

In [ ]:
import os